# HomeWork 5

## Web Scaping from  https://krisha.kz/

### Name: Diana Dalenova
### Group: BDA-1901

I am going to scrape info about apartments for sales in new buildings in Nur-Sultan

In [2]:
#let me download selenium
#!pip install selenium

In [3]:
#let me import necessary packages
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import math
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [4]:
#creating an instance
driver = webdriver.Chrome() 

In [25]:
start = time.time() #i want to record the time
krisha = pd.DataFrame(columns=['OwnerName', 'Address', 'Complex', 'Square', 'Rooms', 'Phone', 'Price', 'Description'])
data = [] #array for the future dataframe
i=1 #setting 1, because we start with page 1, not 0
for i in range(11):
    driver.get('https://krisha.kz/prodazha/kvartiry/nur-sultan/?das[novostroiki]=1&page={}'.format(i)) #putting i instead of {}
    page1 = driver.page_source
    soup1 = BeautifulSoup(page1, 'lxml') #just parsing
    flats = soup1.find_all('a', class_ = 'a-card__title') #getting links that are stored in the titles
    
    for j in range(len(flats)):
        driver.get('https://krisha.kz{}'.format(flats[j]['href'])) #putting href of flats instead of {}
        page2 = driver.page_source
        
        #if you open source code of the website, you can find script named jsdata
        #data that i need is stored in variable "data" 
        #i found it useful, because there i can, for example, get price without tenge sign
        jsdata = driver.execute_script('return data') 

        array = {
            'OwnerName':jsdata['advert']['ownerName'],
            'Address':jsdata['advert']['addressTitle'],
            'Complex':jsdata['advert']['complexName'],
            'Square':jsdata['advert']['square'],
            'Rooms':jsdata['advert']['rooms'],
            'Phone':jsdata['adverts'][0]['phones'][0], #for phones we have many phone numbers in jsdata, so we take first one
            'Price':jsdata['advert']['price'],
            'Description':jsdata['advert']['title']
        }
        data.append(array) 
    df = pd.DataFrame(data)
        
print('That was scraped by', time.time()-start, 'seconds')

That was scraped by 451.6558139324188 seconds


In [26]:
krisha = krisha.append(df, ignore_index=True)

In [27]:
krisha

,OwnerName,Address,Complex,Square,Rooms,Phone,Price,Description
0,Асем Тас 2,Туран 38/1,Асем Тас 2,101.10,3,+7 778 017 1147,44484000,"3-комнатная квартира, 101.1 м²"
1,Athletic,Туран — Улы Дала,Athletic,79.85,3,+7 778 021 1839,25552000,"3-комнатная квартира, 79.85 м²"
2,Atlant,Сарайшык 2 — Кунаева,Atlant,100.42,3,+7 778 021 1851,44929027,"3-комнатная квартира, 100.42 м²"
3,Клубный дом «Green House»,Н. Хлудова 1,Клубный дом Green House,74.92,2,+7 778 021 1830,31316560,"2-комнатная квартира, 74.92 м²"
4,Сentral Park,Туран 5,Central Park,207.22,4,+7 778 017 1138,97393400,"4-комнатная квартира, 207.22 м²"
...,...,...,...,...,...,...,...,...
215,Жеруйык,Кенесары — Кумисбекова,Жеруйык,83.42,3,+7 778 017 1130,23774700,"3-комнатная квартира, 83.42 м²"
216,Cosmic City,Туран 89,Cosmic City,105.99,3,+7 778 021 1869,25437600,"3-комнатная квартира, 105.99 м²"
217,Cosmic City,Туран 89,Cosmic City,98.98,3,+7 778 021 1869,23755200,"3-комнатная квартира, 98.98 м²"
218,Budapest,Кабанбай батыра 59 — Хусейн бен Талала,Budapest,55.61,2,+7 778 017 1134,14458600,"2-комнатная квартира, 55.61 м²"


In [28]:
krisha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OwnerName    220 non-null    object 
 1   Address      220 non-null    object 
 2   Complex      220 non-null    object 
 3   Square       220 non-null    float64
 4   Rooms        220 non-null    object 
 5   Phone        220 non-null    object 
 6   Price        220 non-null    object 
 7   Description  220 non-null    object 
dtypes: float64(1), object(7)
memory usage: 13.9+ KB


In [29]:
#we do not have null values, but dtypes are object, so let's convert some of them into int or float
krisha['Rooms'] = krisha['Rooms'].astype(float)
krisha['Price'] = krisha['Price'].astype(float)
krisha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OwnerName    220 non-null    object 
 1   Address      220 non-null    object 
 2   Complex      220 non-null    object 
 3   Square       220 non-null    float64
 4   Rooms        220 non-null    float64
 5   Phone        220 non-null    object 
 6   Price        220 non-null    float64
 7   Description  220 non-null    object 
dtypes: float64(3), object(5)
memory usage: 13.9+ KB


In [30]:
#saving into csv
krisha.to_excel("krisha.xlsx") 